In [1]:
import os
import glob
import numpy

from pypeit.metadata import PypeItMetaData


In [2]:
file_list = glob.glob(os.path.join(os.environ['PYPEIT_DEV'],
                                   'RAW_DATA/Keck_LRIS_red/multi*/*.fits*'))

In [3]:
fitstbl = PypeItMetaData('keck_lris_red', file_list=file_list)

[WARNING] :: metadata.py 137 _build() - Columns are missing.
[WARNING] :: metadata.py 182 _build() - ut keyword not in header. Setting to None
[INFO]    :: metadata.py 208 _build() - Successfully loaded headers for file:
             /Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_1200_9000_d680/r170816_0009.fits
[WARNING] :: metadata.py 182 _build() - ut keyword not in header. Setting to None
[INFO]    :: metadata.py 208 _build() - Successfully loaded headers for file:
             /Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_1200_9000_d680/r170816_0018.fits
[WARNING] :: metadata.py 182 _build() - ut keyword not in header. Setting to None
[INFO]    :: metadata.py 208 _build() - Successfully loaded headers for file:
             /Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_1200_9000_d680/r170816_0019.fits
[WARNING] :: metadata.py 182 _build() - ut keyword not in header. Set

[WARNING] :: metadata.py 182 _build() - ut keyword not in header. Setting to None
[INFO]    :: metadata.py 208 _build() - Successfully loaded headers for file:
             /Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560/r170320_2075.fits.gz
[WARNING] :: metadata.py 182 _build() - ut keyword not in header. Setting to None
[INFO]    :: metadata.py 208 _build() - Successfully loaded headers for file:
             /Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560/r170320_2076.fits.gz
[WARNING] :: metadata.py 182 _build() - ut keyword not in header. Setting to None
[INFO]    :: metadata.py 208 _build() - Successfully loaded headers for file:
             /Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560/r170320_2084.fits.gz
[WARNING] :: metadata.py 182 _build() - ut keyword not in header. Setting to None
[INFO]    :: metadata.py 208 _build() - Su

In [4]:
_ = fitstbl.get_frame_types(flag_unknown=True)

[INFO]    :: metadata.py 701 get_frame_types() - Typing files
[WARNING] :: keck_lris.py 130 check_frame_type() - Cannot determine if frames are of type standard.
[INFO]    :: metadata.py 756 get_frame_types() - Couldn't identify the following files:
[INFO]    :: metadata.py 758 get_frame_types() - r170816_0018.fits
[INFO]    :: metadata.py 758 get_frame_types() - r170816_0019.fits
[INFO]    :: metadata.py 758 get_frame_types() - r170816_0020.fits
[INFO]    :: metadata.py 758 get_frame_types() - r170816_0021.fits
[INFO]    :: metadata.py 758 get_frame_types() - r170816_0022.fits
[INFO]    :: metadata.py 770 get_frame_types() - Typing completed!


In [5]:
cfgs = fitstbl.unique_configurations(ignore_frames=['bias', 'dark'])

In [6]:
cfgs

{'A': {'dispname': '1200/9000', 'dichroic': '680', 'decker': 'E570.fil'},
 'B': {'dispname': '400/8500', 'dichroic': '560', 'decker': 'c17_60L.'}}

In [7]:
fitstbl.set_configurations(cfgs)

In [8]:
fitstbl.set_calibration_groups(global_frames=['bias', 'dark'])

In [9]:
fitstbl

directory,filename,instrume,idname,time,utc,ut,ra,dec,airmass,binning,decker,dichroic,target,exptime,hatch,dispname,dispangle,wavecen,spectrograph,lampstat01,lampstat02,lampstat03,lampstat04,lampstat05,lampstat06,lampstat07,lampstat08,lampstat09,lampstat10,lampstat11,lampstat12,filter1,NAXIS01,CCDGEOM,CCDNAME01,NAXIS02,NAXIS03,CCDNAME02,NAXIS04,frametype,framebit,configuration,calib
str98,str20,str13,str4,float64,str11,str4,str11,str11,float64,str3,str8,str3,str8,int64,str6,str9,float64,float64,str4,str3,str3,str3,str3,str3,str3,str3,str3,str3,str3,str3,str3,str5,int64,str27,str4,int64,int64,str4,int64,str15,int16,str4,object
/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_1200_9000_d680,r170816_0009.fits,keck_lris_red,dark,57981.122341,02:56:10.22,None,15:20:00.00,+45:00:00.0,1.41309236,"1,1",E570.fil,680,unknown,3,closed,1200/9000,47.82505798,8609.26074219,LRIS,on,on,on,on,on,on,on,on,off,off,off,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,arc,1,A,0
/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_1200_9000_d680,r170816_0018.fits,keck_lris_red,dark,57981.138398,03:19:17.60,None,15:20:00.00,+45:00:00.0,1.41309236,"1,1",E570.fil,680,unknown,10,closed,1200/9000,47.82505798,8609.26074219,LRIS,off,off,off,off,off,off,off,off,off,off,off,on,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,0,A,0
/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_1200_9000_d680,r170816_0019.fits,keck_lris_red,dark,57981.140168,03:21:50.55,None,15:20:00.00,+45:00:00.0,1.41309236,"1,1",E570.fil,680,unknown,10,closed,1200/9000,47.82505798,8609.26074219,LRIS,off,off,off,off,off,off,off,off,off,off,off,on,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,0,A,0
/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_1200_9000_d680,r170816_0020.fits,keck_lris_red,dark,57981.141755,03:24:07.60,None,15:20:00.00,+45:00:00.0,1.41309236,"1,1",E570.fil,680,unknown,10,closed,1200/9000,47.82505798,8609.26074219,LRIS,off,off,off,off,off,off,off,off,off,off,off,on,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,0,A,0
/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_1200_9000_d680,r170816_0021.fits,keck_lris_red,dark,57981.14334,03:26:24.00,None,15:20:00.00,+45:00:00.0,1.41309236,"1,1",E570.fil,680,unknown,10,closed,1200/9000,47.82505798,8609.26074219,LRIS,off,off,off,off,off,off,off,off,off,off,off,on,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,0,A,0
/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_1200_9000_d680,r170816_0022.fits,keck_lris_red,dark,57981.144926,03:28:41.64,None,15:20:00.00,+45:00:00.0,1.41309236,"1,1",E570.fil,680,unknown,10,closed,1200/9000,47.82505798,8609.26074219,LRIS,off,off,off,off,off,off,off,off,off,off,off,on,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,0,A,0
/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_1200_9000_d680,r170816_0045.fits,keck_lris_red,dark,57981.186772,04:28:57.09,None,15:20:00.00,+45:00:00.0,1.41309237,"1,1",long_1.5,680,unknown,0,closed,1200/9000,48.42325211,8740.19921875,LRIS,off,off,off,off,off,off,off,off,off,off,off,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,bias,2,None,"0,1"
/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_1200_9000_d680,r170816_0046.fits,keck_lris_red,dark,57981.188259,04:31:05.54,None,15:20:00.00,+45:00:00.0,1.41309237,"1,1",long_1.5,680,unknown,0,closed,1200/9000,48.42325211,8740.19921875,LRIS,off,off,off,off,off,off,off,off,off,off,off,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,bias,2,None,"0,1"
/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_1200_9000_d680,r170816_0047.fits,keck_lris_red,dark,57981.189748,04:33:14.19,None,15:20:00.00,+45:00:00.0,1.41309237,"1,1",long_1.5,680,unknown,0,closed,1200/9000,48.

In [ ]:
fitstbl.match_to_science(fitstbl.par['calibrations'], fitstbl.par['rdx']['calwin'], fitstbl.par['fluxcalib'], setup=True)

In [ ]:
fitstbl

In [ ]:
print(numpy.log2(fitstbl['sci_ID'].data).astype(int))
indx = numpy.log2(fitstbl['sci_ID'].data).astype(int) == 5
print(fitstbl['filename'][indx])

In [ ]:
fitstbl.find_frames('trace', sci_ID=5)

In [ ]:
fitstbl['frametype']